**Approach: Query Expansion + Word2Vec + Rochio + gamma= 0**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
df=pd.read_csv('drive/My Drive/dataset.csv')

In [0]:
import pandas as pd
dfq=pd.read_csv('drive/My Drive/dataset_query_exp.csv')

In [0]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [5]:
sent=[]
for i in range(125511):
  if(i%10000==0):
    print(i)
  tokens=df["Tokens"][i][1:len(df["Tokens"][i])-2]
  tokens_list=tokens.split(', ')
  for i in range(len(tokens_list)):
    tokens_list[i]=tokens_list[i][1:len(tokens_list[i])-1]
  sent.append(tokens_list)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [6]:
print(len(sent))

125511


**Word embedding using SkipGram Technique**

Window size is taken as 2 , Dimension: 100

In [0]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
model = Word2Vec(sent, size=100, window=2, min_count=1,sg=1) #using skipgram technique

In [0]:
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
model=Word2Vec.load("drive/My Drive/word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.train(sent,total_examples=125511,epochs=4)

(129914225, 131597408)

In [8]:
sent_q=[]
for i in range(50):
  if(i%5==0):
    print(i)
  tokens=dfq["Tokens"][i][1:len(dfq["Tokens"][i])-1]
  tokens_list=tokens.split(', ')
  for i in range(len(tokens_list)):
    tokens_list[i]=tokens_list[i][1:len(tokens_list[i])-1]
  sent_q.append(tokens_list)

0
5
10
15
20
25
30
35
40
45


In [0]:
model.wv["reason"] #Sample word vector of 100 dimensions

array([-0.29024836,  0.1606922 ,  0.52632654, -0.1637964 ,  0.03614322,
       -0.49315777, -0.22694942,  0.2694871 ,  0.45510322,  0.24887332,
        0.09076   , -0.19231062,  0.21659519, -0.1800657 ,  0.37272257,
       -0.30173963,  0.46155885, -0.3380218 , -0.34793428,  0.30021107,
        0.13928668, -0.37237358,  0.67735475, -0.41021997, -0.17133668,
       -0.7945007 ,  0.29764795, -0.26591083, -0.15248977, -0.35347056,
       -0.3059114 , -0.00502704,  0.12856673,  0.06625283, -0.15359287,
        0.263658  , -0.2502284 , -0.31511596,  0.1963704 ,  0.3382178 ,
        0.44306117,  0.13616435, -0.00379117, -0.18723637, -0.06207887,
        0.14326905, -0.5428333 ,  0.58212566, -0.34781802,  0.04669193,
       -0.7346095 , -0.31885302, -0.06603304,  0.14635526,  0.04139513,
       -0.14362943,  0.2978085 ,  0.15617462,  0.3285208 ,  0.76740634,
       -0.05319729, -0.20620589,  0.18436311,  0.21690753,  0.55841845,
        0.14592141,  0.25610292,  0.14462425,  0.16700995, -0.26

In [9]:
# calculating vectors for each doc

import numpy as np
doc_rep=np.zeros((125511,100),dtype='float32')
for i in range(125511):
  docs=sent[i]
  no_of_words=len(docs)
  word_rep=[]
  for x in docs:
    word_rep.append(model.wv[x]);
  word_rep_arr=np.array(word_rep)
  if(i%10000==0):
    print(word_rep_arr.shape)
    print(i)
  doc_rep[i]=np.average(word_rep_arr,axis=0)


(111, 100)
0
(177, 100)
10000
(167, 100)
20000
(178, 100)
30000
(157, 100)
40000
(79, 100)
50000
(280, 100)
60000
(170, 100)
70000
(138, 100)
80000
(303, 100)
90000
(98, 100)
100000
(199, 100)
110000
(411, 100)
120000


In [10]:
print(doc_rep.shape)

(125511, 100)


In [11]:
# calculating vectors for each query 
import numpy as np
query_rep=np.zeros((50,100),dtype='float32')
init=np.zeros((1,100),dtype='float32') # for words not present in trained model
p=0;
for i in range(50):
  #if(i==11 or i==25):
   # continue
  query=sent_q[i]
  no_of_words=len(query)
  q_rep=[]
  for x in query:
    try:
      q_rep.append(model.wv[x]);
    except:
      q_rep.append(init)
  q_rep_arr=np.array(q_rep)
  print(i)
  query_rep[p]=np.average(q_rep_arr,axis=0)
  p=p+1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [0]:
print(query_rep.shape)

(50, 100)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cos_lib = cosine_similarity(query_rep,doc_rep)

In [0]:
import pandas as pd
df3=pd.read_csv('drive/My Drive/file_names.csv')

import pandas as pd
df4=pd.read_csv('drive/My Drive/relevant_docs.csv')

Note: **Threshold has been taken as 0.83**

In [14]:
dist=[]
rel_docs=[] # to store relev
actual_rel_docs=[] # actual rel
rel_docs_ids=[] # to store indices of rel doc vectors
p=0;
for k in range(0,50):
  dist=[]
  a=[] # to store relev
  b=[] # actual rel
  ai=[] # to store indices of relev
  for i in range(len(cos_lib[k])):
    dist.append((i,cos_lib[k][i]))

  dist=sorted(dist,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist[i][1]>=0.83 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a.append(df3["File_Name"][dist[i][0]].split('\\')[-1])
    ai.append(dist[i][0])
    i=i+1;
  
  print("-----------")
  
  print("Actual Relevant docs: ")
  mdf=(df4[df4["Id"]==k+76])
  #print(mdf)
  #print(len(mdf))
  
  for i in range(len(mdf)):
    b.append(mdf["Doc_Name"][p])
    #print(mdf["Doc_Name"][p])
    p=p+1;
  rel_docs.append(a)
  actual_rel_docs.append(b) # actual rel
  rel_docs_ids.append(ai)
  print("------------------------------------")

Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
------------------------------------
Processing...
-----------
Actual Relevant docs: 
-----

In [15]:
matches=0
for i in range(50):
  print(len(actual_rel_docs[i])," ",len(set(rel_docs[i]) & set(actual_rel_docs[i])))

5   4
20   6
3   2
4   0
8   3
6   5
23   21
9   3
3   2
22   15
7   7
9   7
22   20
47   26
5   5
12   0
38   0
19   15
25   2
11   4
3   3
22   16
5   0
3   1
21   6
13   12
17   0
7   3
8   6
13   13
4   0
14   14
11   7
10   7
11   2
16   0
17   0
14   0
8   5
19   2
10   5
1   1
10   7
7   0
12   11
21   0
20   17
19   9
7   1
13   0


In [16]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  295


In [0]:
print(list(set(rel_docs[0]) & set(actual_rel_docs[0])))

['1070602_nation_story_7865940.utf8', '1070603_nation_story_7869357.utf8', '1040901_nation_story_3702283.utf8', '1070611_nation_story_7906812.utf8', '1070530_nation_story_7849973.utf8']


In [0]:
import pandas as pd
df5=pd.read_csv('drive/My Drive/file_names_new.csv')

In [23]:
dr_rev=np.zeros((50,100),dtype='float32')
ndr_rev=np.zeros((50,100),dtype='float32')
for i in range(50):
  dr=[]
  ndr=[]
  print("processing")
  matched_files=list(set(rel_docs[i]) & set(actual_rel_docs[i]))
  for k in range(125511):
    if(df5["File_Name"][k] in matched_files):
      dr.append(doc_rep[k]) # storing relevant vectors 
    else:
      ndr.append(doc_rep[k]) # storing non relevant vectors
  
  dr_arr=np.array(dr) # conversion into array
  ndr_arr=np.array(ndr)
  dr_rev[i]=np.average(dr_arr,axis=0) #finding average of all relevant doc vectors
  ndr_rev[i]=np.average(ndr_arr,axis=0)

processing
processing
processing
processing


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing


In [0]:
print(dr_rev[4])

[-0.30261573 -0.17261827  0.31989288 -0.17712684 -0.10863154 -0.317651
  0.11150265  0.17496566  0.06815441  0.07477763  0.09860824  0.29610214
  0.1285724  -0.16312607  0.16676687 -0.12959039  0.17175451 -0.41663122
 -0.14525405 -0.10312333 -0.08668302 -0.18375048  0.46141505  0.3374409
 -0.18595535 -0.4641924  -0.15313874 -0.10973386  0.11181398 -0.38842502
 -0.09740677  0.01939033 -0.15007903  0.28319854 -0.05891808  0.3110803
 -0.421836   -0.3134853   0.16279826  0.17262237  0.37702936 -0.0370237
 -0.06865855  0.24487019 -0.00902329  0.02460778 -0.12523423  0.30941385
 -0.2596913   0.15122403 -0.36215204 -0.24956112 -0.06604018 -0.28103238
  0.06277902 -0.09090327 -0.00172408 -0.01235844  0.16783574  0.26614302
 -0.03147031 -0.05432379  0.26207146 -0.06184213  0.27167445 -0.06452016
  0.15028414  0.0099738   0.40149826 -0.4491867  -0.09249078 -0.15874343
 -0.00188263 -0.29630286 -0.03253992  0.39026156 -0.08439225  0.09628262
  0.05680417  0.22478385 -0.3041623  -0.06260191 -0.0767

In [0]:
beta=0.75
gamma=0
mod_q=np.zeros((50,100),dtype='float32')
for i in range(50):
  if(len(list(set(rel_docs[i]) & set(actual_rel_docs[i])))>0): # checking no. of matches > 0 or not
    total_rel=len(list(set(rel_docs[i]) & set(actual_rel_docs[i])));
    total_non_rel=125511-total_rel
    term=beta/total_rel;
    mod_q[i]=np.add(query_rep[i],term*dr_rev[i])
    #mod_q[i]=np.subtract(mod_q[i],(gamma/total_non_rel)*ndr_rev[i])
  else:
    mod_q[i]=query_rep[i]
    #mod_q[i]=np.subtract(mod_q[i],(gamma/total_non_rel)*ndr_rev[i])

In [0]:
print(query_rep[1])

[-1.38826743e-01 -2.89725542e-01  8.91866311e-02 -1.62501857e-01
 -1.79269329e-01 -2.09240884e-01  8.73422623e-02  2.11404130e-01
 -1.28635168e-01  8.95918161e-02 -1.47978514e-01  1.16049066e-01
  1.10434815e-01 -1.41472727e-01  1.28401235e-01 -2.02566028e-01
 -7.58671835e-02 -4.02699381e-01 -1.28056323e-02 -8.33550319e-02
  2.91001201e-02  3.56562026e-02  6.01795614e-01  1.06458232e-01
 -2.87494659e-01 -2.08319739e-01  5.17137870e-02 -2.68226445e-01
  1.28063008e-01 -1.86850145e-01 -2.50460416e-01  2.74546780e-02
 -2.07499298e-03  2.00916484e-01 -1.63578197e-01  3.40943635e-01
 -3.15326452e-01 -2.97441214e-01  6.55261101e-03  1.70179129e-01
  1.70377567e-01 -9.92521569e-02 -5.62905110e-02  1.34052247e-01
  1.06473885e-01  4.97432053e-02 -4.99299169e-02  1.11714430e-01
 -7.33399391e-02  1.38946235e-01 -4.99737918e-01 -2.25056544e-01
  2.74080075e-02 -2.45268688e-01  2.34320149e-01 -8.40348676e-02
  2.38386691e-02  6.02361560e-02 -4.47600223e-02  3.02614152e-01
  1.92013962e-04  1.86042

In [0]:
print(mod_q[1])

[-0.17665371 -0.31130284  0.12917325 -0.18464272 -0.19284827 -0.24894726
  0.10128009  0.23327483 -0.12011587  0.09893902 -0.13565248  0.15306184
  0.12650636 -0.16186349  0.1492471  -0.21876483 -0.05439787 -0.45477828
 -0.03096239 -0.09624545  0.01826474  0.01268739  0.65947247  0.14863834
 -0.31073907 -0.26634377  0.03257144 -0.28194317  0.14203976 -0.23540327
 -0.26263627  0.02987847 -0.02083487  0.2363163  -0.17094296  0.37982866
 -0.36805594 -0.3366269   0.02690239  0.19175692  0.21750623 -0.10388012
 -0.06487283  0.16466102  0.10534597  0.05281918 -0.0655842   0.15039116
 -0.10580135  0.15784924 -0.54500693 -0.2562517   0.01915298 -0.28039774
  0.24216753 -0.09539778  0.02362316  0.05869135 -0.02378055  0.33588204
 -0.00374177  0.17925176  0.42487815  0.0129588   0.23077154  0.05188191
  0.12409978 -0.09586538  0.32094204 -0.19834356 -0.0699192  -0.03772284
  0.2962604  -0.14053774 -0.05566746  0.34138152 -0.05070833  0.0566531
  0.18026353 -0.03892665 -0.28194442 -0.26942486 -0.

In [0]:
#finding relevant with the modified query

from sklearn.metrics.pairwise import cosine_similarity
cos_lib2 = cosine_similarity(mod_q,doc_rep)

In [27]:
print(cos_lib2.shape)

(50, 125511)


In [0]:
dist2=[]
for i in range(len(cos_lib2[0])):
  dist2.append((i,cos_lib2[0][i]))

dist2=sorted(dist,key=lambda x:x[1],reverse=True)

In [28]:
rel_docs2=[] # to store relev
p=0;
for k in range(0,50):
  dist2=[]
  a2=[] # to store relev
  b2=[] # actual rel
  for i in range(len(cos_lib2[k])):
    dist2.append((i,cos_lib2[k][i]))

  dist2=sorted(dist2,key=lambda x:x[1],reverse=True)
  print("Processing...");
  i=0;
  while(dist2[i][1]>=0.83 and i<125511):
   # print(dist[i]," ",df3["File_Name"][dist[i][0]].split('\\')[-1])
    a2.append(df3["File_Name"][dist2[i][0]].split('\\')[-1])
    i=i+1;
  
  rel_docs2.append(a2)

  print("------------------------------------")

Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
------------------------------------
Processing...
-----------------

In [29]:
matches=0
for i in range(50):
  matches=matches+len(set(rel_docs2[i]) & set(actual_rel_docs[i]))

print("Total Matches: ",matches)

Total Matches:  381


As we can see, no. of matches increased from 295 to 381 after implementing Rocchio Algorithm

In [30]:
print("Average Precison: ", matches/125511)
print("Average Recall: ", matches/653)

Average Precison:  0.0030355905060114254
Average Recall:  0.5834609494640123
